In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch
from torch import nn
from torch.utils import data
from d2l import torch as d2l

关于下标
1. 0~N-1：琪琪的属性嵌入、匹配度、满意度
2. 1~N：whc做的那两个dict文件
3. 真实ID：导出文件格式（题目要求）

In [2]:
# ID映射表
# df_per = pd.read_csv('./user_dict',sep='\t')
# df_rec = pd.read_csv('./jd_dict',sep='\t')

In [2]:
# 匹配度、满意度，这里的序号是0~N-1
df_ppd = pd.read_csv('./p-u-rating.txt',sep=' ')
df_myd = pd.read_csv('./u-p-rating.txt',sep=' ')
df_ppd.head(), df_myd.head()

(   position  user  rating
 0         0    94       6
 1         0   165       6
 2         0   168       6
 3         1    94       6
 4         1   165       6,
    user  position  myd
 0   425       295    8
 1   425      1504    8
 2   425      1515    8
 3   425      1542    8
 4   425      1543    8)

In [3]:
def is_fw(pid):
  # return not pid in idx
  # print(f'is_fw({pid})')
  val = fw_list.loc[pid,'是否为废人']
  return val==1

fw_list = pd.read_csv('./fw.txt',sep=' ',index_col=0)
fw_list.head()

,是否为废人
下标,
0,0
1,1
2,1
3,1
4,1


In [4]:
alive = []
for i in range(10877):
  if is_fw(i):
    pass
  else:
    alive.append(i)

In [5]:
for i in range(10877):
  print(is_fw(i),end=' ')

False True True True True True True False True True False True False False False False True True True True False True False False False False False False False False False False False True True False False False False False False False False False False False False False False False True True False True False False True False True False False True True False False False False True False False False False False True False False False False True False False False False False False False False True False False False False False False False True False False True True False False False False False False False False False False False True False False False True False False False False False False True False False True False False False False False False False False False True True True False False False False False False False False False False True False False False False False False False False False False False False False False False False False False False False True False True False Fa

In [6]:
# per_id = {}
# for i in df_per.index:
#   per_id[df_per.iloc[i,0]] = df_per.iloc[i,1]
# rec_id = {}
# for i in df_rec.index:
#   rec_id[df_rec.iloc[i,0]] = df_rec.iloc[i,1]

In [7]:
for i in range(74):
  print(f'{i} ',end='')

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 

In [8]:
# df_map.to_csv('./per_job_map.csv',header=['job','person','label'],index=None)

In [9]:
# 属性嵌入，这里的序号是0~N-1
a_emb_job = pd.read_csv('./attribute_embedding_position.txt',sep=' ',index_col=0,header=None,na_values=[])
a_emb_per = pd.read_csv('./attribute_embedding_user.txt',sep=' ',index_col=0,header=None,na_values=[])
a_emb_per.index = [alive[i] for i in range(868)]
print(a_emb_job.shape, a_emb_per.shape)
a_emb_per.head()

(1575, 74) (868, 74)


,1,2,3,4,5,6,7,8,9,10,...,65,66,67,68,69,70,71,72,73,74
0,-0.724494,-0.559137,-0.300785,0.008680,-0.170450,0,0,0.0,1,0,...,-0.160764,0.035278,0.060169,0.088794,-0.306723,0.137945,0.316116,0.4042,0.127718,0.047178
7,-0.724494,-0.721122,-0.890788,4.934121,3.222017,0,0,0.0,1,0,...,-0.160764,0.035278,0.060169,0.088794,-0.306723,0.137945,0.316116,0.4042,0.127718,0.047178
10,1.741550,-0.802114,-0.890788,2.832705,7.543888,0,0,0.0,0,1,...,-0.160764,0.035278,0.060169,0.088794,-0.306723,0.137945,0.316116,0.4042,0.127718,0.047178
12,-0.724494,-0.397152,-0.448285,0.080217,-0.179261,0,0,0.0,1,1,...,-0.160764,0.035278,0.060169,0.088794,-0.306723,0.137945,0.316116,0.4042,0.127718,0.047178
13,-0.724494,3.247509,6.041750,0.025324,-0.260802,0,0,0.0,1,1,...,-0.160764,0.035278,0.060169,0.088794,-0.306723,0.137945,0.316116,0.4042,0.127718,0.047178


先算PU网络，得到匹配度

In [10]:
# no_fw = pd.read_csv('./no_fw.csv',index_col=0)
# def per_id(x):
#   return no_fw.iloc[x,0]

# for i in range(10):
#   print(per_id(i),end=' ')

In [11]:
# 结构嵌入，这里的序号是0~N-1
d_emb_job = pd.read_csv('./position_pu.csv',index_col=0,header=0,na_values=[])
d_emb_per = pd.read_csv('./user_pu.csv',index_col=0,header=0,na_values=[])
d_emb_per.index = [alive[i] for i in range(868)]
print(d_emb_job.shape, d_emb_per.shape)
d_emb_per.head()

(1575, 128) (868, 128)


,1,2,3,4,5,6,7,8,9,10,...,119,120,121,122,123,124,125,126,127,128
0,0.106909,-0.046381,0.227989,0.011383,0.049010,0.013847,0.011298,-0.075828,-0.220346,-0.297584,...,0.305304,0.071091,0.004234,0.159778,0.208874,0.261583,0.304411,-0.178296,-0.006043,0.402940
7,0.176486,0.125179,0.170178,0.010691,-0.109280,0.048260,0.103159,-0.049660,-0.293907,-0.443231,...,0.141347,0.170352,0.099519,0.360286,0.144341,0.294506,0.422864,-0.047133,-0.079090,0.512646
10,0.070882,0.197714,0.028229,0.098573,-0.222974,-0.022671,-0.054216,-0.003531,-0.285085,-0.488723,...,-0.069236,0.208382,0.019828,0.413162,0.065986,0.334274,0.341294,0.069452,-0.118390,0.537937
12,0.186596,-0.067015,0.246331,0.018256,0.053443,0.091744,-0.038790,-0.039274,-0.235857,-0.276473,...,0.248856,0.136002,0.001937,0.131124,0.154355,0.266727,0.293040,-0.170457,0.010407,0.351705
13,0.169177,0.083607,0.280480,0.036082,-0.073978,0.088527,0.002985,-0.015185,-0.162121,-0.377745,...,0.212814,0.159757,0.102400,0.217491,0.118156,0.201218,0.407281,-0.038142,-0.057384,0.484031


In [12]:
d_emb_per.mean()

1      0.069535
2     -0.003523
3      0.013527
4      0.105477
5      0.119242
         ...   
124    0.272369
125    0.285738
126   -0.159624
127    0.090348
128    0.140246
Length: 128, dtype: float64

In [13]:
# a = np.array([[1,2,3],[4,5,6]])
# b = np.array([[-1,-2],[-3,-4]])
a = a_emb_per.values
b = d_emb_per.values
X = np.concatenate([a,b],axis=1)

c = a_emb_job.values
d = d_emb_job.values
Y = np.concatenate([c,d],axis=1)
X.shape, Y.shape

((868, 202), (1575, 202))

In [14]:
import sklearn
from sklearn.decomposition import PCA
pca=PCA(n_components=64)
pca.fit(X)
Xpca = pca.transform(X) #进行降维
pca.fit(Y)
Ypca = pca.transform(Y)

In [15]:
Xpca.shape, Ypca.shape

((868, 64), (1575, 64))

In [16]:
pca_per = pd.DataFrame(Xpca, index=[alive[i] for i in range(868)])
pca_job = pd.DataFrame(Ypca)
pca_per.shape,pca_job.shape

((868, 64), (1575, 64))

In [17]:
def get_input(pid, rid):
  per = pca_per.loc[pid].values
  job = pca_job.loc[rid].values
  all = np.concatenate([per,job],axis=0)
  return all

In [18]:
# pca_per.to_csv('./pca_per.csv',header=None)
# pca_job.to_csv('./pca_job.csv',header=None)
# pca_per = pd.read_csv('./pca_per.csv',index_col=0,header=None)

In [19]:
features = []
labels = []
# get_input(7,1)

In [20]:
df_ppd.shape

(388183, 3)

In [21]:
for i in tqdm(range(df_ppd.shape[0])):
  row = df_ppd.loc[i]
  pid = row['user']
  rid = row['position']
  label = row['rating']
  if is_fw(pid):
    continue
  # print(pid,rid,label)
  feature = get_input(pid,rid)
  features.append(feature)
  labels.append(label)
  

100%|██████████| 388183/388183 [01:25<00:00, 4557.92it/s]


In [22]:
features = np.array(features)
labels = np.array(labels)
features = torch.Tensor(features)
labels = torch.Tensor(labels).reshape(-1,1)
features.shape,labels.shape

(torch.Size([388183, 128]), torch.Size([388183, 1]))

In [23]:
features, labels

(tensor([[ 1.5127, -0.5813, -0.8024,  ..., -0.0703,  0.0079, -0.1002],
         [ 1.0280, -0.0294, -0.2958,  ..., -0.0703,  0.0079, -0.1002],
         [ 1.1276, -0.1380, -0.5021,  ..., -0.0703,  0.0079, -0.1002],
         ...,
         [ 1.7114, -0.2400, -0.4685,  ..., -0.0276,  0.0258, -0.0200],
         [ 2.0525, -0.4133, -0.3817,  ..., -0.0276,  0.0258, -0.0200],
         [ 0.2351, -0.0658, -0.5743,  ..., -0.0276,  0.0258, -0.0200]]),
 tensor([[6.],
         [6.],
         [6.],
         ...,
         [5.],
         [5.],
         [5.]]))

In [75]:
np.median(features)

-0.0008885674

In [24]:
class DS:
  def __init__(self,data_arrays):
    self.data = data_arrays

  def __getitem__(self,index):
    return self.data[0][index],self.data[1][index]
  
  def __len__(self):
    return self.data[0].size()[0]

def load_array(data_arrays, batch_size, is_train=True):
  # Array to Iter
  dataset = data.TensorDataset(*data_arrays)
  return data.DataLoader(dataset, batch_size, shuffle=is_train)
    

In [25]:
batch_size = 64

In [26]:
train_iter = load_array((features,labels), batch_size)
# test_iter = load_array((features[train_size:train_size+test_size],labels[train_size:train_size+test_size]), batch_size, False)

In [27]:
labels.shape

torch.Size([388183, 1])

In [66]:
net = nn.Sequential(nn.Linear(128,32),
                    nn.Sigmoid(),
                    nn.Linear(32,1)
                    )

In [28]:
net = torch.load('./DNN-4000.pt')

In [67]:
def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, std=0.01)

net.apply(init_weights)

Sequential(
  (0): Linear(in_features=128, out_features=32, bias=True)
  (1): Sigmoid()
  (2): Linear(in_features=32, out_features=1, bias=True)
)

In [68]:
loss = nn.MSELoss()
# loss = nn.CrossEntropyLoss(reduction='none')
trainer = torch.optim.Adam(net.parameters(), lr=0.01)

In [53]:
total_epochs = 0

In [69]:
num_epochs = 200
looper = tqdm(range(num_epochs))
for epoch in looper:
    total_epochs += 1
    for X, y in train_iter:
        # print(X.shape, y.shape)
        l = loss(net(X) ,y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    # print(f'epoch {epoch + 1}, loss {l:f}')
    looper.set_description(f'epoch {total_epochs}, loss {l:f}')

epoch 402, loss 0.129940: 100%|██████████| 200/200 [37:29<00:00, 11.25s/it]


In [83]:
print(net(torch.Tensor(get_input(77,1111)))) # None
print(net(torch.Tensor(get_input(0,1)))) # None
print(net(torch.Tensor(get_input(8031,1038)))) # 2
print(net(torch.Tensor(get_input(94,0)))) # 6
# net.state_dict()

tensor([8.2627], grad_fn=<AddBackward0>)
tensor([5.8987], grad_fn=<AddBackward0>)
tensor([2.1007], grad_fn=<AddBackward0>)
tensor([6.8922], grad_fn=<AddBackward0>)


In [64]:
net[2].weight

Parameter containing:
tensor([[ 0.1781,  0.1368,  0.8412,  0.4497,  0.1604, -0.1248,  0.3549, -0.2746,
          0.2133,  0.2675,  0.3607,  0.2020,  0.2258,  0.1848,  0.1926,  0.1966,
          0.5588,  0.1551, -0.3235,  0.1985,  0.1819,  0.5139,  0.1870,  0.1578,
          0.4102,  0.2142,  0.1941,  0.5321,  0.4127,  0.1205,  0.8987,  0.2716]],
       requires_grad=True)

In [71]:
torch.save(net,'Sigmoid-200.pt')